<a href="https://colab.research.google.com/github/tariqmassaoudi/ONIVIA/blob/main/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd
import numpy as np


In [122]:
df=pd.read_csv('/content/sample_data/dataopti.csv')

In [123]:
df.head()

,farmer,Besoin,risk
0,1,200,0.0000
1,2,300,0.0025
2,3,100,0.0050
3,4,150,0.0075
4,5,150,0.0100


In [124]:
pip install -U "scipy==1.4.*" "pulp==2.1"


Requirement already up-to-date: scipy==1.4.* in /usr/local/lib/python3.6/dist-packages (1.4.1)
Requirement already up-to-date: pulp==2.1 in /usr/local/lib/python3.6/dist-packages (2.1)


In [125]:
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpAffineExpression

In [132]:
#define parameters 
ls = df.Besoin.to_list() #loan size
T = df.risk.to_list() #risk rate
S = 2500 #budget

# Create the model
model = LpProblem(name="credit-problem", sense=LpMinimize)

# Initialize the decision variables: x is integer, y is continuous
y = [LpVariable(name=f"{i}", cat="Binary") for i in range(len(df))]

# Add the constraints to the model
model += (LpAffineExpression([(y[i],ls[i])for i in range (len(df))]) == S)
model += (LpAffineExpression([(y[i],ls[i])for i in range (len(df))]) >= 0.8*S)

# Add the objective function to the model
Tls = [T[i]*ls[i] for i in range(len(df))]
model += LpAffineExpression([(y[i],Tls[i]) for i in range(len(df))])

# Solve the problem
status = model.solve()

In [133]:
model.variables()

[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 3, 4, 5, 6, 7, 8, 9]

In [134]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")


0: 1.0
1: 1.0
10: 0.0
11: 1.0
12: 1.0
13: 0.0
14: 1.0
15: 0.0
16: 0.0
17: 0.0
18: 0.0
19: 0.0
2: 1.0
20: 0.0
3: 1.0
4: 1.0
5: 1.0
6: 1.0
7: 1.0
8: 1.0
9: 1.0


In [135]:
#create results' dictionnary
dict={}
for var in model.variables():
  dict[var.name] = var.value()

#convert the dictionnary into a dataset
results=pd.DataFrame(dict.items(),columns=['ID','decision'])

#sort and match index/value
results.ID = pd.to_numeric(results.ID)
results.set_index('ID', inplace=True)
results = results.sort_index()

#concat df with results into final decisions ( same results in excel)
data = pd.concat([df, results], axis=1)
data

,farmer,Besoin,risk,decision
0,1,200,0.0000,1.0
1,2,300,0.0025,1.0
2,3,100,0.0050,1.0
3,4,150,0.0075,1.0
4,5,150,0.0100,1.0
5,6,100,0.0125,1.0
6,7,200,0.0150,1.0
7,8,200,0.0175,1.0
8,9,200,0.0200,1.0
9,10,300,0.0225,1.0
